## Sentiment Analysis - Feedforward NN with CBOW

Extend the following code to use a continuous BOW representation (e.g., sum of embeddings of all words in the sentence) as instance representation. 

Hint: use `LookupParameters`, see [DyNet reference](https://dynet.readthedocs.io/en/latest/tutorials_notebooks/API.html)

In [ ]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [ ]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]

def read_dataset(filename):
    """
    Read data and covert to indices
    """
    with open(filename, "r") as f:
        for line in f:
            tag, sentence = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in sentence.split(" ")], t2i[tag])

In [ ]:
# Read in the data
train = list(read_dataset("data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i) # freeze vocab
dev = list(read_dataset("data/classes/dev.txt"))

vocab_size = max(w2i.values()) + 1 # OOV

ntags = len(t2i)
print(vocab_size, ntags)

In [ ]:
# print the label of the first instance
train[0][1]

In [ ]:
len(train[0][0])

In [ ]:
len(train[1][0])

In [ ]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.SimpleSGDTrainer(model)

In [ ]:
# Define the model
EMB_SIZE = 64
hidden_size = 100
W_emb = None # Word embeddings TODO: instantiate your lookup parameters

W1 = model.add_parameters((hidden_size, EMB_SIZE))
b1 = model.add_parameters(hidden_size)

W_sm = model.add_parameters((ntags, hidden_size))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [ ]:
# A function to calculate scores for one value
def calc_scores(words):
    dy.renew_cg()
    word_vecs = None   ## TODO: look up embeddings
    cbow = dy.esum(word_vecs) ## sum over the current word embeddings
    h = dy.tanh(dy.parameter(W1) * cbow + dy.parameter(b1))
    score = dy.parameter(W_sm) * h + dy.parameter(b_sm)
    return score

In [ ]:
for ITER in range(10):
    # Perform training
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    for words, tag in train:
        my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
        train_loss += my_loss.value()
        my_loss.backward()
        trainer.update()
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
    # Perform testing
    test_correct = 0.0
    for words, tag in dev:
        predict = dy.softmax(calc_scores(words)).npvalue().argmax()
        if predict == tag:
            test_correct += 1
    print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))
